In [36]:
import metadrive
from metadrive.envs.base_env import BaseEnv
from metadrive.obs.observation_base import ObservationBase
from metadrive.constants import DEFAULT_AGENT
from metadrive.utils import Config
from metadrive.manager.scenario_map_manager import ScenarioMapManager
from metadrive.manager.base_manager import BaseManager
from metadrive.scenario.scenario_description import ScenarioDescription
from metadrive.component.vehicle_navigation_module.trajectory_navigation import TrajectoryNavigation



In [2]:
import os
import pickle

path = "~/myworkspace/scenarionet/waymo/waymo_0/sd_waymo_v1.2_1b254a843598dc5f.pkl"
path = os.path.expanduser(path)

with open(path, "rb") as f:
    data = pickle.load(f)

In [3]:
print(data)

{'id': '1b254a843598dc5f', 'version': 'v1.2', 'length': 199, 'tracks': {'1309': {'type': 'VEHICLE', 'state': {'position': array([[ 3266.2827 ,  -872.73413,  -279.16458],
       [ 3266.2854 ,  -873.95374,  -279.1758 ],
       [ 3266.282  ,  -875.1924 ,  -279.17035],
       [ 3266.2852 ,  -876.4076 ,  -279.1843 ],
       [ 3266.2803 ,  -877.6149 ,  -279.19165],
       [ 3266.2783 ,  -878.81696,  -279.20538],
       [ 3266.2751 ,  -880.03503,  -279.22345],
       [ 3266.2676 ,  -881.23334,  -279.22437],
       [ 3266.2764 ,  -882.43024,  -279.22644],
       [ 3266.2705 ,  -883.6547 ,  -279.24396],
       [ 3266.2722 ,  -884.8817 ,  -279.2566 ],
       [ 3266.2703 ,  -886.08984,  -279.26755],
       [ 3266.2664 ,  -887.2927 ,  -279.26785],
       [ 3266.2603 ,  -888.49365,  -279.28418],
       [ 3266.2605 ,  -889.6792 ,  -279.29263],
       [ 3266.2593 ,  -890.85144,  -279.30865],
       [ 3266.2546 ,  -892.05396,  -279.31885],
       [ 3266.2468 ,  -893.22766,  -279.33206],
       [ 3266.

In [50]:
class InMemoryScenarioDataManager(BaseManager):
    def __init__(self, scenario):
        super().__init__()
        self.current_scenario = scenario
    
    def get_scenario(self, i, should_copy=False):
        return self.current_scenario

class ReplayEnv(BaseEnv):
    @classmethod
    def default_config(cls) -> Config:
        config = super().default_config()
        config.update({
            "start_seed": 0,
            "num_scenarios": 1,
            "start_scenario_index": 0,
            "no_map": False,
            "store_map": False,
            "need_lane_localization": True,
            "vehicle_config": dict(
                lidar=dict(num_lasers=120, distance=50),
                lane_line_detector=dict(num_lasers=0, distance=50),
                side_detector=dict(num_lasers=12, distance=50),
                show_dest_mark=True,
                navigation_module=TrajectoryNavigation,
            ),
            "max_lateral_dist": 4
        })
        return config

    def __init__(self, config, scenario):
        super().__init__(config)
        self.scenario = scenario

    def done_function(self, vehicle_id: str):
        return False, {}

    def cost_function(self, vehicle_id: str):
        return 0, {}
    
    def reward_function(self, vehicle_id: str):
        return 0, {}

    def _get_observations(self):
        return {DEFAULT_AGENT: self.get_single_observation()}
    
    def setup_engine(self):
        self.engine.register_manager("agent_manager", self.agent_manager)
        self.engine.register_manager("map_manager", ScenarioMapManager())
        self.engine.register_manager("data_manager", InMemoryScenarioDataManager(self.scenario))


In [52]:
env = ReplayEnv(config={"use_render": True, "manual_control": True}, scenario=ScenarioDescription(data))

In [53]:
env.reset()
for i in range(1000):
    env.step([0, 1])

Known pipe types:
  glxGraphicsPipe
(1 aux display modules not yet loaded.)
:task(warning): Creating implicit AsyncTaskChain default for AsyncTaskManager TaskManager
show_dest_mark and show_line_to_dest are not supported in TrajectoryNavigation


In [51]:
env.close()